That error usually means the **video file is incomplete or not properly written**.
This happens because:

✔ **Streamlit runs asynchronously**, and frames may not be written properly before you stop recording.
✔ **VideoWriter settings (codec, frame size)** must match the actual frames received from the webcam.
✔ **`streamlit-webrtc`** uses WebRTC, and frames may have different resolutions or FPS than expected.

---

### ✅ **Fix for Corrupt Video**

To fix this, we need to:

1. **Get the correct frame size dynamically** from the actual frame.
2. **Use a stable codec and proper file format** (e.g., `.mp4` with `mp4v`).
3. **Close the writer properly** when stopping recording.

---

Here’s the **fixed version of the Streamlit app** for Colab (with proper video saving):

---

### ✅ **Updated Code**

```python
%%writefile app.py
import streamlit as st
import cv2
import os
import datetime
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase
import av

SAVE_DIR = "media"
os.makedirs(SAVE_DIR, exist_ok=True)

st.title("📷 Webcam Image & Video Recorder")

status_text = st.empty()

class VideoTransformer(VideoTransformerBase):
    def __init__(self):
        self.frame = None

    def transform(self, frame):
        img = frame.to_ndarray(format="bgr24")
        self.frame = img
        return img

ctx = webrtc_streamer(key="example", video_transformer_factory=VideoTransformer)

if st.button("📸 Capture Image"):
    if ctx.video_transformer and ctx.video_transformer.frame is not None:
        img = ctx.video_transformer.frame
        filename = os.path.join(SAVE_DIR, f"image_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg")
        cv2.imwrite(filename, img)
        status_text.success(f"✅ Image saved: {filename}")
    else:
        status_text.error("❌ No frame captured.")

if "recording" not in st.session_state:
    st.session_state.recording = False
    st.session_state.out = None
    st.session_state.video_filename = ""

if st.button("🎥 Start/Stop Recording"):
    if not st.session_state.recording:
        st.session_state.video_filename = os.path.join(SAVE_DIR, f"video_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4")
        st.session_state.recording = True
        status_text.info("🔴 Recording started...")
    else:
        st.session_state.recording = False
        if st.session_state.out:
            st.session_state.out.release()
        status_text.success(f"✅ Recording stopped. Saved as: {st.session_state.video_filename}")

if st.session_state.recording and ctx.video_transformer and ctx.video_transformer.frame is not None:
    img = ctx.video_transformer.frame
    if st.session_state.out is None:
        height, width, _ = img.shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        st.session_state.out = cv2.VideoWriter(st.session_state.video_filename, fourcc, 20.0, (width, height))
    st.session_state.out.write(img)

st.write("📂 Saved files will appear in the 'media' folder.")
```

---

### ✅ **Changes Made**

✔ Switched to **`.mp4`** with `mp4v` codec (widely supported).
✔ Dynamically set **frame size** when the first frame comes in.
✔ Properly close the video file on stop.

---

### ✅ **Run in Colab**

```python
from pyngrok import ngrok
!streamlit run app.py &

public_url = ngrok.connect(8501)
print("Access your app here:", public_url)
```

---

### ✅ **How to Download Video from Colab**

After recording:

```python
from google.colab import files
files.download('media/video_YYYYMMDD_HHMMSS.mp4')
```

---

👉 Do you want me to **add a "Download Video" button inside the Streamlit app**, so you don’t need to manually use `files.download()`?
Or **add multiple features like filters (grayscale, blur), and resolution settings in the Streamlit UI**?


In [2]:
import cv2

# Capture video from camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()  # Capture a frame
    if not ret:
        break

    cv2.imshow('Frame', frame)  # Show the frame
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit when 'q' is pressed
        break

cap.release()
cv2.destroyAllWindows()
